In [ ]:
# import os
# os._exit(00)

In [ ]:
## environment preparation
##============================

%reset -f
%connect_info #might be used for "jupyter-console"

In [ ]:
## Hint. (crutch / quick workaround)
## HOW TO RESTART THIS NOTEBOOK TO COLLECT A FEW SAMPLES
##============================

## To restart this notebook for multiple data collection there are two options:

## option A
##-----------
## uncomment the 1st cell of this notebook 
## with the code 
##   import os
##   os._exit(00)

## option B
##-----------
## use Kernel -> "Restart & Run All"
## instead of just "Run All"


## This will ensure that the paths are correct and No error will appear.
## Yes, this is an crutch (quick workaround), 
## but so far there was not found better solution to ensure 
## that data would be always recorded to a correct dicrectory

In [ ]:
## Hint
##============================

### check the status of the port (linux version)
# sudo ls -la /dev/ttyACM*  
### must be 666=rw all
# !groups your_os_user_name
### must be group tty

In [ ]:
## Hint
##============================

### Use in terminal
# sudo chmod 666 /dev/ttyACM0
### or RUN here
# !xfce4-terminal -e "sudo chmod 666 /dev/ttyACM0 "

In [ ]:
## Hint
##============================

##### Try this if previos cell does not work
###-----------------------
### Use in terminal
# sudo usermod -a -G tty $USER
### or RUN here
# !xfce4-terminal -e "sudo usermod -a -G tty $USER"

In [ ]:
#######################################################
#######################################################

# Reading is done via Arduino Uno via post ttyACMO
# The reading frequecny is 10Hz (100ms)
# the 1st 7..10 samples should be thrown away 
# bcoz of bad data format

# for each reading change:
#     dir_2_save = "run_A1/"
#     label = "sampleX"
#     add_label = "1"
#     env_condition = "condition1"
#     N_of_readings = 20 #=2sec

#######################################################
#######################################################

In [ ]:
## Import libraries
##============================

import serial
from datetime import datetime
import time
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
## Settings to Change
##============================

dir_2_save = "run_A1/"

label = "sampleX1"
add_label = "1"
env_condition = "condition1"

reading_duration = 0.2 # in min, 0.2min=12sec

In [ ]:
## Additional Settings 
##============================

N_of_readings = 10*60*reading_duration #10Hz 60sec

serial_port = '/dev/ttyACM0'
baud_rate = 9600

In [ ]:
## Adjusting directory to save data
##============================

path_ipynb = os.getcwd()
path_base = path_ipynb + '/../../data/'
path_add_rdbk = 'readback/'
path_work = path_base + path_add_rdbk
os.chdir(path_work)

try:
    os.makedirs(dir_2_save)
except:
    print('folder already exists')
    pass
        
os.chdir(dir_2_save)
pathW = os.getcwd()
print(pathW)

In [ ]:
## Data Acquisition 
##============================

pbar = tqdm(total=N_of_readings)
GGS_list = []
init_time = round(time.time(),3)*1000

# Main loop for data acquisition
## If it breaks first of all try to restart
with serial.Serial(serial_port, baud_rate) as ser:
    for cc1 in tqdm(range(10)):
        line_z = ser.readline();
        line_dec_z = line_z.decode("utf-8")
    while len(GGS_list) < N_of_readings:
        line = ser.readline();
        line_dec = line.decode("utf-8") #ser.readline returns a binary, convert to string
        lst0 = line_dec.split(",")
        lst1 = [int(x) for x in lst0]
        if len(lst1)==6:
            GGS_list += [[int(round(time.time(),3)*1000-init_time)] + lst1]
            pbar.update(1)
            plt.plot

In [ ]:
## Acqusition result
##============================

GGS_list[0:6]

In [ ]:
## Convert to DataFrame
##============================

GGS_df = pd.DataFrame(GGS_list, columns=
                      ["time_ms", "B102NO2", "B302C2H5OH", "B502VOC", "B702CO", 
                       "TdegC", "RH"])

In [ ]:
## Save to File
##============================

filename_to_save = "./%s_%s_%s_%s.csv" % (label, add_label, env_condition, 
                                          datetime.now().strftime("%Y%m%d_%H%M%S"))

with open(filename_to_save, 'w') as output_file:
    GGS_df.to_csv(output_file, index=False)
    print("Saved to file: " + filename_to_save)
    print("Under the path: " + pathW)

In [ ]:
## Visualisation of the collected data
##============================

plt.figure(figsize=(10,5))
plt.plot(GGS_df.time_ms, GGS_df.drop('time_ms', axis=1))
plt.legend(GGS_df.drop('time_ms', axis=1).columns.to_list())
plt.tick_params(axis='x', colors='red')
plt.tick_params(axis='y', colors='red')
plt.grid()

In [ ]:
## The collected data overview
##============================

GGS_df.head()